In [ ]:
import numpy as np
from macpp import MACPPAgent, MACPP
from maf import Grid
import matplotlib.pyplot as plt
import json
from matplotlib import colors
import time
import os
import pandas as pd
%load_ext autoreload
%autoreload 2

In [ ]:
l,b=50,50
# grid=np.zeros((l,b))
# grid[int(l/2),:]=-1
# grid[int(l/2),int(b/2)]=0
# grid[int(l/2),int(b/2)+1]=0
# plt.imshow(grid)

In [ ]:
def run_simulation(num_agents):
    g=Grid('map_50x50.json')
    grid=g.return_grid()
    m=MACPP(grid,4)
    agents=[]
    n=num_agents

    #randomize start points in every experiment
    #idxes are numpy coordinates. convert them to cartesian
    idxes=np.argwhere(grid==0)
    start=np.random.randint(len(idxes))

    #reverse convert from numpy coordinates to cartesian coordinates
    x, y = m.reverse_convert(idxes[start][0],idxes[start][1])
    print(x,y)

    #print(m.n*x+y)
    #this is the 1D coordinate of numpy coordinates
    crds=np.ones(n)*(m.n*x+y)
    for i in range(n):
        #x and y are cartesian coordinates
        agents.append(MACPPAgent(x,y,i,m))
        
    coverage_percentage=[]

    i=0
    while m.marked_visited()<1:
        if agents[i%n].state():
            crds[i%n]=agents[i%n].next(crds)
            # to record x and y coodinates of agents
            # x,y=agents[i%n].x,agents[i%n].y
        i+=1
        if i%n==0:
            print(i%n,m.covered(),end=' | ')
            coverage_percentage.append(m.marked_visited())
    # return variables to save: coverage percentage, crds, visit_freq
    return coverage_percentage, crds, m.visit_frequency

In [ ]:
num_runs=20
num_agents=20

for num_agent in range(2, num_agents+1):
	for run in range(num_runs):
		#get the returned data and store it in csv
		coverage_percentage, crds, visit_freq = run_simulation(num_agent)

		df=pd.DataFrame(coverage_percentage)
		df.to_csv(r'data/coverage_percentage.csv', mode='a')

		df=pd.DataFrame(crds)
		df.to_csv(r'data/agent_area_dist.csv', mode='a')

		df=pd.DataFrame(visit_freq)
		df.to_csv(r'data/visit_freq.csv', mode='a')

		tempCounter=0
		for row in visit_freq:
			for colVal in row:
				if colVal>1:
					tempCounter+=1

		overlapped_cells_percentage=tempCounter/(l*b)

		df=pd.DataFrame([overlapped_cells_percentage])
		df.to_csv(r'data/overlapped_cells_percentage.csv', mode='a')
		print(f'finished {run} run(s) for {num_agent} agents')